In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import pylab as pl
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("data.csv")
df = pd.DataFrame(data) # Converting data to Panda DataFrame
df.head() #gives statistics about the columns of the dataframe

,dataSet,Input,instructionCount,ilp32,ilp256,ilp2048,ilp65536,memops,ctrlops,intops,...,lbdiv32,lbdiv64,lbdiv128,lbdiv256,lbdiv512,lbdiv1024,probsize,Class,CPU Time,GPU Time
0,MVT_8192: 51-63,8192,26233453,6.498181,13.011978,13.078701,15.976306,0.820424,0.051388,0.846086,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,74,Y,11.144549,3.324654
1,MVT_16384: 51-63,16384,104658385,6.499089,13.005995,13.039291,14.338086,0.820468,0.051335,0.846120,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,148,Y,5.509123,1.766470
2,MVT_20000: 51-63,20000,156024013,6.157181,13.004910,13.032163,14.076138,0.820476,0.051325,0.846126,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,181,Y,1.165420,0.658309
3,syrk_m_2048: 59-70,2048,23223871910,6.749968,9.031183,9.332883,14.100434,0.666661,0.074120,0.666661,...,0.000285,0.000570,0.001140,0.002280,0.004560,0.009120,18,Y,15.701999,14.617724
4,syrk_m_1024: 59-70,1024,2996927698,6.999141,9.398286,10.028998,15.750233,0.642840,0.071516,0.678563,...,0.001104,0.002209,0.004417,0.008835,0.017669,0.035338,9,Y,1.954343,1.856112


## Weight all the features by instruction count rather than a separate instruction

In [3]:
X_ = df

Weighted_X = X_[["ilp32",'ilp256','ilp2048','ilp65536','memops','ctrlops','intops','flops',
 'coldref fraction',
 'resuseDist2 fraction',
 'sfp',
 'dfp',
 'noconflict',
 'broadCast',
 'coalesced',
 'shMemBw',
 'gMemBw',
 'blocks',
 'pages',
 'lipRate',
 'mulf',
 'divf',
 'specialFn',
 'lbdiv16',
 'lbdiv32',
 'lbdiv64',
 'lbdiv128',
 'lbdiv256',
 'lbdiv512',
 'lbdiv1024']].multiply(df["instructionCount"], axis="index")


Weighted_X.shape

(92, 30)

## Splitting Data to Training set and Test set 0.1%

In [4]:
Y_train = df['Class']

#seperate traindata and test data here
X_train, X_test, Y_train, Y_test = train_test_split(Weighted_X,Y_train ,test_size=0.1)


## Reading Test Data from unseen Data (Guaranteed_)

In [5]:
%%script false
data_test = pd.read_csv("test.csv")
df_test = pd.DataFrame(data_test) # Converting data to Panda DataFrame
df_test.head() #gives statistics about the columns of the dataframe

In [6]:
%%script false
X_test = df_test
Weighted_XTest = X_test[["ilp32",'ilp256','ilp2048','ilp65536','memops','ctrlops','intops','flops',
 'coldref fraction',
 'resuseDist2 fraction',
 'sfp',
 'dfp',
 'noconflict',
 'broadCast',
 'coalesced',
 'shMemBw',
 'gMemBw',
 'blocks',
 'pages',
 'lipRate',
 'mulf',
 'divf',
 'specialFn',
 'lbdiv16',
 'lbdiv32',
 'lbdiv64',
 'lbdiv128',
 'lbdiv256',
 'lbdiv512',
 'lbdiv1024']].multiply(df_test["instructionCount"], axis="index")


Weighted_XTest.head()

## Class Distribution

In [7]:
yes = len(data[data.Class == "Y"])
No = len(data[data.Class == "N"])

print yes
print No

69
23


In [8]:
#Y_train = df['Class']

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)


#Y_test = df_test['Class']
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

print len(X_train)
print len(X_test)

82
10


## Ensembled Bagging Decison Tree Classifier

In [9]:
baggingClassifier = ensemble.BaggingClassifier(n_estimators=11,bootstrap_features=True,n_jobs=-1)
baggingClassifier.fit(X_train,Y_train)

BaggingClassifier(base_estimator=None, bootstrap=True,
         bootstrap_features=True, max_features=1.0, max_samples=1.0,
         n_estimators=11, n_jobs=-1, oob_score=False, random_state=None,
         verbose=0, warm_start=False)

## Cross validating the model

In [10]:
scores = cross_val_score(baggingClassifier, X_train, Y_train)
print scores

[0.75       0.85185185 0.77777778]


In [11]:
prediction = baggingClassifier.predict(X_test)
print prediction
print Y_test

['Y' 'N' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'N']
['Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'N']


In [12]:
accuracy_score(Y_test, prediction)

0.8

## Random Forest Classifier


In [13]:
#similiar to ensembling techniques
#use majority vote from the forest
#can handle missing values in the dataset
#samples are used with replacement

randomForest = RandomForestClassifier(n_estimators=11,n_jobs=-1,random_state=0)
scores = cross_val_score(randomForest, X_train, Y_train)
print scores

#for n in range(5):
#    strat_k_fold = StratifiedKFold(n_splits=5, shuffle=True)
#    scores = cross_val_score(randomForest, X_train, Y_train, cv=strat_k_fold)
#    print scores

[0.82142857 0.81481481 0.81481481]


In [14]:
randomForest.fit(X_train,Y_train)
prediction = randomForest.predict(X_test)

print prediction
print Y_test

['Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y']
['Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'N']


In [15]:
accuracy_score(Y_test, prediction)

0.7

In [16]:
importances = pd.DataFrame({'feature':Weighted_X.columns,'importance':np.round(randomForest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')

importances.head(32)

,importance
feature,
coldref fraction,0.122
pages,0.091
lbdiv256,0.088
ilp2048,0.065
blocks,0.062
memops,0.059
shMemBw,0.053
lipRate,0.051
lbdiv128,0.043


## Executing Recursive Feature Selection to choose important features <- Check

## Nearest Neighbor Method

In [17]:
neighbor = KNeighborsClassifier(n_neighbors=11,weights='distance',algorithm='auto')
scores = cross_val_score(neighbor, X_train, Y_train)
print scores

[0.67857143 0.81481481 0.74074074]


In [18]:
neighbor.fit(X_train,Y_train)
prediction = neighbor.predict(X_test)

print prediction
print Y_test

['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y']
['Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'N']


In [19]:
accuracy_score(Y_test, prediction)

0.6

## SVM Classifier 

In [20]:
svmClassifier = SVC(kernel='rbf')
scores = cross_val_score(svmClassifier, X_train, Y_train)
print scores


[0.75       0.81481481 0.81481481]


In [21]:
svmClassifier.fit(X_train,Y_train)
prediction = svmClassifier.predict(X_test)

print prediction
print Y_test

['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y']
['Y' 'Y' 'N' 'Y' 'Y' 'Y' 'Y' 'N' 'N' 'N']


In [22]:
accuracy_score(Y_test, prediction)

0.6